<a href="https://colab.research.google.com/github/Siddharth02003/CS6910_assignments/blob/main/CS6910_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import fashion_mnist
import numpy as np
import matplotlib.pyplot as plt
import copy 
import random
import tensorflow as tf

In [ ]:
!pip install wandb -qqq
import wandb

In [ ]:
def load_dataset():
    (X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()
    return {
            'X_train': X_train,
            'Y_train': Y_train,
            'X_test': X_test,
            'Y_test': Y_test
        }

In [ ]:
class_labels={0:'T-shirt',1:'Trouser',2:'Pullover',3:'Dress',4:'Coat',5:'Sandal',6:'Shirt',7:'Sneaker',8:'Bag',9:'Ankle_Boot'}

def label_to_name(label):
  if enumerate(label):
    l_names=[]
    for l in label:
      l_names.append(class_labels[l])
    return l_names
  else:
    return class_labels[label]

In [ ]:
!wandb login --relogin
entity_name="siddharth-s"

project_name="FODL_Assignment_1"

In [ ]:
def log_images():
  images=[]
  labels=[]
  dataset=load_dataset()
  X_train=dataset['X_train']
  y_train=dataset['Y_train']
  wandb.init(entity=entity_name,project=project_name, name="log_images")
  for i in range(100):
    if len(labels)==10:
      break
    if class_labels[y_train[i]] not in labels:
      images.append(X_train[i])
      labels.append(class_labels[y_train[i]])
  wandb.log({"Images": [wandb.Image(img, caption=caption) for img, caption in zip(images,labels)]})

In [ ]:
class activation: 
  def __init__(self,a,x,activation):
    self.a=a
    self.x=x
    self.activation=activation

  def sigmoid(self,a):
    try:
      return (1.0/(1.0+np.exp(-a)))
    except:
      print("error")

  def relu(self,a):
    return (np.maximum(0,a))

  def tanh(self,a):
    return np.tanh(a)

  def softmax(self,x):
    try:
      return(np.exp(x)/np.sum(np.exp(x)))
    except:
      print("error")

  def sigmoid_derivative(self,x):
    return self.sigmoid(x)*(1-self.sigmoid(x))

  def tanh_derivative(self,x):
    return 1.0 -self.tanh(x)**2

  def relu_derivative(self,x):
    return 1. * (x>0)
     
  def softmax_derivative(self,x):
    return self.softmax(x) * (1-self.softmax(x))

  def derivative(self,a):
    if self.activation == "sigmoid":
      return self.sigmoid_derivative(a)
    elif self.activation == "tanh":
      return self.tanh_derivative(a)
    elif self.activation == "relu":
      return self.relu_derivative(a)

In [ ]:
class weights:
  def __init__(self,layers):
    self.layers=layers

  def Xavier(self,layers):
    params = {}
    for i in range(1,len(layers)):
       norm_xav=np.sqrt(6)/np.sqrt(layers[i]+layers[i-1])
       params["w"+str(i)]=np.random.randn(layers[i],layers[i-1])*norm_xav
       params["b"+str(i)]=np.zeros((layers[i],1))
    return params

  def Random(self,layers):
    params = {}
    for i in range(1,len(layers)):
       params["w"+str(i)]=0.01*np.random.randn(layers[i],layers[i-1])
       params["b"+str(i)]=0.01*np.random.randn(layers[i],1)
    return params

  def weight_init(self,layers, init_type = "random"):
    params={}
    if(init_type=="xavier"):
      params = self.Xavier(layers)
    elif(init_type=="random"):
      params = self.Random(layers)
    else:
      print("invalid activation function")
    return params

In [ ]:
class loss:
  def __init__(self,y,y_hat):
    self.y_hat=y_hat
    self.y=y

  def squared_loss(self,y,y_hat):
   error = np.sum(((y - y_hat)**2) / (2 * len(y)))
   return error

  def CrossEntropy(self,y,y_hat):
   error = - np.sum( np.multiply(y , np.log(y_hat)))/len(y)
   return error

  def loss_calc(self,loss_name, y, y_hat, lambd, layers, parameters):
   error=0
   if(loss_name == "squared_loss"):
     error=self.squared_loss(y, y_hat)
   elif(loss_name == "cross_entropy"):
     error= self.CrossEntropy(y, y_hat)
   regularized_error = 0.0
   for i in range(len(layers)-1, 0, -1):
     regularized_error += (np.sum(parameters["w"+str(i)]))**2
   regularized_error = error + ((lambd/(2*len(y)))*(regularized_error))
   return regularized_error

   def calc_accuracy(self,y_hat, y):
    acc=0.0   
    for x in range(len(y_hat)):
      if(y_hat[x].argmax()==y[x].argmax()):
        acc+=1
    acc=acc/len(y)
    return(acc*100)

    def calc_test_accuracy(self,y_hat, y):
     acc=0.0
     for i in range(len(y_hat)):
      if(y_hat[i]==y[i]):
        acc+=1
     acc=acc/len(y)
     return(acc*100)